In [11]:
import pandas as pd
import numpy as np
# from fuzzywuzzy import fuzz, process
from rapidfuzz import fuzz, process, utils
import re
from tqdm import tqdm

def clean_bank_name(name):
    """
    Clean and standardize bank names for better matching.
    """
    if pd.isna(name):
        return ""
    
    # Convert to string and lowercase
    name = str(name).lower()
    
    # Remove common suffixes and prefixes
    remove_terms = [
        'mitigation bank', 'bank', 'mb', 'conservation bank', 'cb', 
        'in-lieu fee program', 'ilf', 'program', 'umbrella', 
        'conservation area', 'preserve', 'restoration'
    ]
    
    for term in remove_terms:
        name = name.replace(term, '')
    
    # Remove special characters and extra spaces
    name = re.sub(r'[^a-z0-9\s]', '', name)
    name = re.sub(r'\s+', ' ', name).strip()
    
    return name

def match_bank_names(withdrawals_df, banks_df, threshold=80):
    """
    Match bank names between withdrawals and banks dataframes using fuzzy matching.
    
    Parameters:
    -----------
    withdrawals_df : pandas DataFrame
        DataFrame containing withdrawal records
    banks_df : pandas DataFrame
        DataFrame containing bank records
    threshold : int
        Minimum similarity score to consider a match (0-100)
        
    Returns:
    --------
    pandas DataFrame
        Original withdrawals DataFrame with matched bank names and scores
    """
    # Clean bank names in both dataframes
    withdrawals_df['Clean_Name'] = withdrawals_df['Name'].apply(clean_bank_name)
    banks_df['Clean_Name'] = banks_df['Name'].apply(clean_bank_name)
    print("Cleaned names")
    
    # Create a dictionary of unique clean bank names
    unique_bank_names = banks_df['Clean_Name'].unique()
    print(unique_bank_names)
    print(withdrawals_df['Clean_Name'])
    
    # Function to find best match for each withdrawal
    best_matches = {}
    def find_best_match(name):
        if pd.isna(name) or name == "":
            return pd.Series({'Matched_Bank_Name': np.nan, 'Match_Score': 0})

        if name in best_matches:
            return best_matches[name]
        
        best_match = process.extractOne(
            name,
            unique_bank_names,
            scorer=fuzz.token_sort_ratio,
            score_cutoff=threshold,
            # scorer=fuzz.WRatio,
            processor=utils.default_process
        )
        
        if best_match:
            return_val = pd.Series({
                'Matched_Bank_Name': best_match[0],
                'Match_Score': best_match[1]
            })
            best_matches[name] = return_val
            return return_val
        return pd.Series({'Matched_Bank_Name': np.nan, 'Match_Score': 0})
    
    # Apply matching
    match_results = withdrawals_df['Clean_Name'].progress_apply(find_best_match)
    print('Matched results')
    
    # Add results to original dataframe
    result_df = withdrawals_df.copy()
    result_df['Matched_Bank_Name'] = match_results['Matched_Bank_Name']
    result_df['Match_Score'] = match_results['Match_Score']

    # Merge latitude and longitude from banks_df
    lat_lon_df = banks_df[['Clean_Name', 'Latitude', 'Longitude']].drop_duplicates()
    
    # Merge on matched bank names to pull latitude and longitude
    result_df = result_df.merge(lat_lon_df, left_on='Matched_Bank_Name', right_on='Clean_Name', how='left')

    # Drop duplicate 'Clean_Name' column after the merge
    result_df = result_df.drop(columns=['Clean_Name_y'])
    
    # Rename columns for clarity
    result_df = result_df.rename(columns={
        'Clean_Name_x': 'Clean_Name',
        'Latitude': 'Matched_Latitude',
        'Longitude': 'Matched_Longitude'
    })
    
    # Get original bank names and details for matches
    bank_name_map = banks_df.set_index('Clean_Name')['Name'].to_dict()
    result_df['Original_Bank_Name'] = result_df['Matched_Bank_Name'].map(bank_name_map)
    
    return result_df

def analyze_matching_results(matched_df):
    """
    Analyze the results of the matching process.
    """
    total_records = len(matched_df)
    matched_records = matched_df['Matched_Bank_Name'].notna().sum()
    match_rate = (matched_records / total_records) * 100
    
    score_distribution = matched_df['Match_Score'].describe()
    
    print(f"Matching Analysis:")
    print(f"Total Records: {total_records}")
    print(f"Matched Records: {matched_records}")
    print(f"Match Rate: {match_rate:.2f}%")
    print("Score Distribution:")
    print(score_distribution)
    
    # Sample of matches at different score levels
    print("Sample matches at different score levels:")
    for score in [100, 90, 80]:
        sample = matched_df[matched_df['Match_Score'] >= score].head(3)
        print(f"Score >= {score}:")
        print(sample[['Name', 'Original_Bank_Name', 'Match_Score']])


# Load your data
# withdrawals_file = 'withdrawals.csv'  # Replace with your file
# banks_file = 'banks.csv'  # Replace with your file
withdrawals_file = 'Bank and ILF Program Credit Tracking 2024_10_16.csv'
banks_file = 'All banks .xlsx'  # Replace with your file
tqdm.pandas()

try:
    withdrawals_df = pd.read_csv(withdrawals_file)
    banks_df = pd.read_excel(banks_file)
    
    # Perform matching
    matched_results = match_bank_names(withdrawals_df, banks_df)
    
    # Analyze results
    analyze_matching_results(matched_results)
    
    # Save results
    matched_results.to_csv('matched_results.csv', index=False)
    print("Results saved to 'matched_results.csv'")
    
except FileNotFoundError as e:
    print(f"Error: {e}")
    print("Please ensure input files exist and are in the correct location.")


C:\Users\snibb\AppData\Local\Temp\ipykernel_9788\3267924886.py:153: DtypeWarning: Columns (19,31,32,34,35,38,41,53,56,58,60,61,62) have mixed types. Specify dtype option on import or set low_memory=False.
  withdrawals_df = pd.read_csv(withdrawals_file)


Cleaned names
['mud slough' 'muddy creek' 'one horse slough sold out' ...
 'wi washington wisdot willow creek stream and wetland'
 'succor creek pending' 'north fork payette river pending']
0                     virginia aquatic resources trust fund
1                     virginia aquatic resources trust fund
2                     virginia aquatic resources trust fund
3                     virginia aquatic resources trust fund
4                     virginia aquatic resources trust fund
                                ...                        
104109                            port fourchon single user
104110                          buffalo quarry conservation
104111                          buffalo quarry conservation
104112    mn itasca s3734t1785 brink farms phase ii mndo...
104113    mn itasca s3734t1785 brink farms phase ii mndo...
Name: Clean_Name, Length: 104114, dtype: object


100%|██████████| 104114/104114 [00:29<00:00, 3545.83it/s] 

Matched results


Matching Analysis:
Total Records: 106110
Matched Records: 97929
Match Rate: 92.29%
Score Distribution:
count    106110.000000
mean         92.234930
std          26.675547
min           0.000000
25%         100.000000
50%         100.000000
75%         100.000000
max         100.000000
Name: Match_Score, dtype: float64
Sample matches at different score levels:
Score >= 100:
                                     Name  \
7431  Keys Environmental Restoration Fund   
7432  Keys Environmental Restoration Fund   
8520                           Mud Slough   

                           Original_Bank_Name  Match_Score  
7431  ILF Keys Environmental Restoration Fund        100.0  
7432  ILF Keys Environmental Restoration Fund        100.0  
8520                               Mud Slough        100.0  
Score >= 90:
                       Name          Original_Bank_Name  Match_Score
1737  The Conservation Fund  The Conservation Fund/AR-1    93.333333
1738  The Conservation Fund  The Conservation F

In [7]:
matches_df = pd.read_csv('matched_results.csv')

# count number of matches by score > 0
print(len(matches_df[matches_df['Match_Score'] > 0]))

# count number of total rows
print(len(matches_df))



95933
104114


C:\Users\snibb\AppData\Local\Temp\ipykernel_9788\3084049197.py:1: DtypeWarning: Columns (19,31,32,34,35,38,41,53,56,58,60,61,62) have mixed types. Specify dtype option on import or set low_memory=False.
  matches_df = pd.read_csv('matched_results.csv')
